<a href="https://colab.research.google.com/github/edwardLum/work-related/blob/main/clustering_search_terms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

Libraries used:

* **pandas**: Pandas pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language. User guide [here](https://pandas.pydata.org/docs/user_guide/index.**html**)

* **gensim**: Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora. Documentation [here](https://github.com/RaRe-Technologies/gensim/#documentation)

* **sklean**: scikit-learn is a free software machine learning library for the Python programming language.[3] It features various classification, regression and clustering algorithms and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy. User guide [here](https://scikit-learn.org/stable/user_guide.html)



In [20]:
import pandas as pd
import chardet

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec

from google.colab import files

**Files upload**

Choose files to upload and pass them to a relevant list

In [15]:
uploaded = files.upload()

filenames = []

# Upload files:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  filenames.append(fn)

Saving Search terms report - 2023-10-17T184921.753.csv to Search terms report - 2023-10-17T184921.753.csv
User uploaded file "Search terms report - 2023-10-17T184921.753.csv" with length 222554 bytes


In [16]:
print(filenames)

['Search terms report - 2023-10-17T184921.753.csv']


**Detect encoding**

Use the detect method of chardet to detect the encoding of the provided file.

In [17]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

file_path = f"/content/{filenames[0]}"
original_encoding = detect_encoding(file_path)

print(f"Detected encoding: {original_encoding}")

Detected encoding: UTF-16


**Read data**

Create dataframe using the provided csv. Provide:

* the separator the csv uses
* the encoding of the file
* how many rows have to be skipped (if any)
* thousands separator

Remove summary rows if any.

In [ ]:
# Load data
all_terms = pd.read_csv(file_path, sep='\t',
                        encoding=original_encoding,
                        skiprows=2,
                        thousands=',')

**Process data**

Actions:
* Remove summary rows
* Remove unnecessary colums


In [ ]:
# Remove Summary rows
all_terms = all_terms[~all_terms['Search term'].str.startswith('Total: ')]

# Remove unnecessary columns
columns_to_drop = ['Conv. rate', 'CTR', 'Cost / conv.', 'Avg. CPC']
all_terms = all_terms.drop(columns=columns_to_drop)